In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import json

# Read recipe inputs
disk_usage = dataiku.Dataset("disk_usage")
df = disk_usage.get_dataframe()

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   status  1 non-null      object
 1   data    1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [3]:
df

,status,data
0,success,"{""result"":[{""metric"":{""cluster"":""mutu-100"",""ho..."


In [0]:
for key, value in df["data"].items():
    print(value)
    dict = json.loads(value)
dict

In [0]:
df = pd.json_normalize(dict["result"])
df = df.explode('values')
df[['Timestamp','CPU_Usage']] = pd.DataFrame(df['values'].tolist(),index=df.index)
df

In [0]:
df.rename(columns={'metric.hostname': 'Broker', 'metric.dc': 'Data_Center','metric.mountpoint':'Disk'}, inplace=True)
df['Broker_Disk'] = df['Broker'] + df['Disk']
df = df.drop(['metric.cluster', 'metric.job','metric.device','metric.fstype','values','Broker','Disk'], axis=1)
df

In [0]:
list(df['Broker_Disk'].unique())

In [0]:
dfx = pd.pivot_table ( df, index=['Timestamp'], columns = df.groupby(['Timestamp']).cumcount().add(1), values = ['CPU_Usage'], aggfunc = 'sum')
dfx.columns = list(df['Broker_Disk'].unique())
dfx = dfx.reset_index()
dfx

In [0]:
df = dfx

In [0]:
# Write recipe outputs
json = dataiku.Dataset("Json")
json.write_with_schema(df)